# Variable Scope

Variable Scope mechanism은 2개의 함수로 구성되어 있습니다. 

* tf.get_variable(<name>, <shape>, <initializer>): Creates or returns a variable with a given name.
* tf.variable_scope(<scope_name>): Manages namespaces for names passed to tf.get_variable().

tf.get_variable 의 경우, 없으면 만들기 때문에 direct로 특정 값을 보내지 않고, initializer를 passing 합니다. <br>
initializers 는 예를들어 다음의 것들이 있습니다. 

* tf.constant_initializer(value) initializes everything to the provided value,
* tf.random_uniform_initializer(a, b) initializes uniformly from [a, b],
* tf.random_normal_initializer(mean, stddev) initializes from the normal distribution with the given mean and standard deviation.

### 예제 

아래의 예제에서 convolution 2개를 만들고자 합니다. <br>
이때 conv_relu에 사용된 이름들 'weights', 'biases', 등이 동일하기 때문에, conv_relu함수를 2번 사용할수가 없습니다.<br>
이때 이름을 구분짓기 위해서 tf.variable_scope을 사용합니다. 

In [1]:
import tensorflow as tf

In [2]:
input_images = tf.placeholder("float", [None, 5, 32, 32])

def conv_relu(input_layer, kernel_shape, bias_shape):
    weights = tf.get_variable('weights', kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable('biases', bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input_layer, weights, strides=[1,1,1,1], padding='SAME')
    return tf.nn.relu(conv + biases)

def create_model(input_layer):
    with tf.variable_scope('conv1'):
        relu1 = conv_relu(input_layer, [5, 5, 32, 32], [32])
    
    with tf.variable_scope('conv2'):
        relu2 = conv_relu(relu1, [5, 5, 32, 32], [32])
    
    return relu2

model = create_model(input_images)

하지만 2번째 부를때는 ValueError가 일어납니다.<br>
tf.get_variable()은 이미 존재하는 variables들을 감지하고 실수로 공유되지 않도록 막습니다.<br>
때문에 저런 에러가 일어난 것이고, share가 필요하다면 reuse_variables()를 사용합니다. 

In [3]:
create_model(input_images)

ValueError: Variable conv1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-abf77357f526>", line 4, in conv_relu
    weights = tf.get_variable('weights', kernel_shape, initializer=tf.random_normal_initializer())
  File "<ipython-input-2-abf77357f526>", line 11, in create_model
    relu1 = conv_relu(input_layer, [5, 5, 32, 32], [32])
  File "<ipython-input-2-abf77357f526>", line 18, in <module>
    model = create_model(input_images)


In [5]:
with tf.variable_scope('image_filters') as scope:
    model2 = create_model(input_images)
    scope.reuse_variables()
    model3 = create_model(input_images)